In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0


In [66]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
PROJECT_DIR = "/content/drive/Colab Notebooks/cleansed_data.csv"
DATA_FILE_NAME = 'Colab Notebooks/input_data.xlsx'
CLEANSED_FILE_DIR = 'Colab Notebooks/'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

In [0]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding , Activation, Bidirectional, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False,delimiter=',')
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred)

In [72]:
cleansed_data_df = load_doc("/content/drive/My Drive/Colab Notebooks/input.xlsx")
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (8500, 4)


In [73]:
cleansed_data_df.describe()

,Short description,Description,Caller,Assignment group
count,8492,8499,8500,8500
unique,7481,7817,2950,74
top,password reset,the,bpctwhsn kzqsbmtp,GRP_0
freq,38,56,810,3976


In [74]:
cleansed_data_df.head()

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


In [0]:
cleansed_data_df["Short description"].fillna("The", inplace=True)
cleansed_data_df["Description"].fillna("The", inplace=True)

In [0]:
cleansed_data = cleansed_data_df.copy()

In [77]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
def remove_punct(text):
    text = re.sub('[0-9]+|\n|\r|[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', ' ', text)
    return text

In [0]:
import re
cleansed_data['Description_new'] = cleansed_data['Description'].apply(lambda x: remove_punct(x))
cleansed_data['Short description'] = cleansed_data['Short description'].apply(lambda x: remove_punct(x))

In [0]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [0]:
from wordcloud import WordCloud, STOPWORDS

In [82]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stop_words)

{'me', 'yours', 'didn', 'isn', 'a', 'those', 'about', "you've", "aren't", 'when', 'where', 'such', 'through', "won't", 'on', 'shouldn', 'am', 'been', 'weren', 'an', 'down', 't', 'ourselves', 'd', 'into', "couldn't", 'of', 'further', 'ours', 'our', 'under', 'here', 'then', 'if', 'don', 'who', 'theirs', 'whom', "it's", 'more', 'between', 'few', 'have', 'did', 'just', "wouldn't", 'all', 'for', 'you', 'off', "doesn't", 'herself', 'wouldn', 'doesn', 'nor', 'the', 'yourself', 'himself', 'won', "weren't", 'both', 'your', 'up', 'its', "needn't", 'now', 'them', "you'd", 'her', 'does', 'over', 'to', 're', "hadn't", 'was', 'as', 'wasn', 'below', 'and', 'no', "shouldn't", 'itself', 'or', 'haven', 'can', "you're", 'above', 'what', 'will', 'hadn', 'out', "didn't", 'has', 'why', 'this', 'at', 'before', 'so', 'i', 'hers', 'it', 'from', 'how', 'but', "wasn't", 'she', 'these', 'than', 'other', 'most', 'until', 'own', 'too', 'against', 'should', 'some', 'which', 'because', 'having', 'while', "you'll", 't

In [0]:
cleansed_data['Description_new1'] = cleansed_data['Description_new'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cleansed_data['Short description'] = cleansed_data['Short description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [85]:
cleansed_data[['Description','Description_new','Description_new1']].head(10)

,Description,Description_new,Description_new1
0,-verified user details.(employee# & manager na...,verified user details employee manager na...,verified user details employee manager name ch...
1,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,received from hmjdrvpb komuaywn gmail com...,received hmjdrvpb komuaywn gmail com hello tea...
2,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,received from eylqgodm ybqkwiam gmail com...,received eylqgodm ybqkwiam gmail com hi cannot...
3,unable to access hr_tool page,unable to access hr tool page,unable access hr tool page
4,skype error,skype error,skype error
5,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,unable log engineering tool skype
6,event: critical:HostName_221.company.com the v...,event critical HostName company com the val...,event critical HostName company com value moun...
7,ticket_no1550391- employment status - new non-...,ticket no employment status new non employ...,ticket employment status new non employee ente...
8,unable to disable add ins on outlook,unable to disable add ins on outlook,unable disable add ins outlook
9,ticket update on inplant_874773,ticket update on inplant,ticket update inplant


In [0]:

cleansed_data['Description'] = cleansed_data['Description_new1']

In [0]:
import re
text1 = "THIS COMMUNICATION IS INTENDED FOR THE SOLE USE OF THE PERSON TO WHOM IT IS ADDRESSED AND MAY CONTAIN INFORMATION THAT IS PRIVILEGED, CONFIDENTIAL AND EXEMPT FROM DISCLOSURE UNDER APPLICABLE LAW. ANY DISSEMINATION, DISTRIBUTION OR DUPLICATION OF THIS COMMUNICATION BY SOMEONE OTHER THAN THE INTENDED RECIPIENT IS STRICTLY PROHIBITED. IF YOUR RECEIPT OF THIS COMMUNICATION IS IN ERROR, PLEASE NOTIFY THE SENDER AND DELETE THIS COMMUNICATION"
cleansed_data['Description'] = cleansed_data['Description'].apply(lambda x: re.sub(text1,'',x))

In [0]:
text2 = "Select the following link to view the Disclaimer in an alternate language."
cleansed_data['Description'] = cleansed_data['Description'].apply(lambda x: re.sub(text2,'',x))

In [0]:
cleansed_data["SD - DD"] = cleansed_data['Short description'].str.cat(cleansed_data["Description"], sep= ' - ')

In [90]:
print(cleansed_data.columns)
cleansed_data.head(10)

Index(['Short description', 'Description', 'Caller', 'Assignment group',
       'Description_new', 'Description_new1', 'SD - DD'],
      dtype='object')


,Short description,Description,Caller,Assignment group,Description_new,Description_new1,SD - DD
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,verified user details employee manager na...,verified user details employee manager name ch...,login issue - verified user details employee m...
1,outlook,received hmjdrvpb komuaywn gmail com hello tea...,hmjdrvpb komuaywn,GRP_0,received from hmjdrvpb komuaywn gmail com...,received hmjdrvpb komuaywn gmail com hello tea...,outlook - received hmjdrvpb komuaywn gmail com...
2,cant log vpn,received eylqgodm ybqkwiam gmail com hi cannot...,eylqgodm ybqkwiam,GRP_0,received from eylqgodm ybqkwiam gmail com...,received eylqgodm ybqkwiam gmail com hi cannot...,cant log vpn - received eylqgodm ybqkwiam gmai...
3,unable access hr tool page,unable access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr tool page,unable access hr tool page,unable access hr tool page - unable access hr ...
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,skype error,skype error - skype error
5,unable log engineering tool skype,unable log engineering tool skype,eflahbxn ltdgrvkz,GRP_0,unable to log in to engineering tool and skype,unable log engineering tool skype,unable log engineering tool skype - unable log...
6,event critical HostName company com value moun...,event critical HostName company com value moun...,jyoqwxhz clhxsoqy,GRP_1,event critical HostName company com the val...,event critical HostName company com value moun...,event critical HostName company com value moun...
7,ticket employment status new non employee ente...,ticket employment status new non employee ente...,eqzibjhw ymebpoih,GRP_0,ticket no employment status new non employ...,ticket employment status new non employee ente...,ticket employment status new non employee ente...
8,unable disable add ins outlook,unable disable add ins outlook,mdbegvct dbvichlg,GRP_0,unable to disable add ins on outlook,unable disable add ins outlook,unable disable add ins outlook - unable disabl...
9,ticket update inplant,ticket update inplant,fumkcsji sarmtlhy,GRP_0,ticket update on inplant,ticket update inplant,ticket update inplant - ticket update inplant


In [0]:
labelencoder = LabelEncoder()
cleansed_data['Assignment group'] = labelencoder.fit_transform(cleansed_data['Assignment group'])

In [0]:
target_count = cleansed_data['Assignment group'].value_counts()

In [93]:

(target_count<=10).value_counts()

False    49
True     25
Name: Assignment group, dtype: int64

In [94]:
oneHotencoder = OneHotEncoder()

#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X = oneHotencoder.fit_transform(cleansed_data['Assignment group'].values.reshape(-1,1)).toarray()
print(X)
#print(cleansed_data.shape[1])
#To add this back into the original dataframe 
#dfOneHot = pd.DataFrame(X, columns = ["assignment_group_"+str(int(i)) for i in range(cleansed_data.shape[1])]) 
#df = pd.concat([cleansed_data, dfOneHot], axis=1)
#droping the country column 
#df= df.drop(['assignment_group'], axis=1) 
#printing to verify 
#print(df.head())

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
from keras.layers.embeddings import Embedding

from sklearn.model_selection import train_test_split

# Split the data
x_train_keras, x_test_keras, y_train_keras, y_test_keras = train_test_split(cleansed_data['SD - DD'], X, test_size=0.15, shuffle= True)
x_train_keras, x_valid_keras, y_train_keras, y_valid_keras = train_test_split(x_train_keras, y_train_keras, test_size=0.15, shuffle= True)

In [40]:
print(x_train_keras.shape)
print(x_valid_keras.shape)
print(x_test_keras.shape)
print(y_train_keras.shape)
print(y_valid_keras.shape)
print(y_test_keras.shape)

(6141,)
(1084,)
(1275,)
(6141, 74)
(1084, 74)
(1275, 74)


In [0]:
from keras.preprocessing import text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [0]:
max_words = 10000
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = text.Tokenizer(num_words=max_words, char_level=False)
tokenizer.fit_on_texts(x_train_keras)
sequences_train = tokenizer.texts_to_sequences(x_train_keras)
sequences_valid = tokenizer.texts_to_sequences(x_valid_keras)
sequences_test = tokenizer.texts_to_sequences(x_test_keras)

In [43]:
MAX_SEQUENCE_LENGTH = 150

# pad sequences with 0s
X_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
X_valid = pad_sequences(sequences_valid, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)
print('Shape of data test tensor:', X_test.shape)
print('Shape of data validation tensor:', X_valid.shape)

Shape of data tensor: (6141, 150)
Shape of data test tensor: (1275, 150)
Shape of data validation tensor: (1084, 150)


In [44]:
type(x_train_keras)

pandas.core.series.Series

In [45]:
x_train_keras.head()

1987    reset passwords jvpkulxw ovuweygj using passwo...
5446                ad account locked - ad account locked
5347    network outage turkey turkey site hard since e...
4397    vip printer driver update - vip printer driver...
2675    cannot access \\HostName \engineering applicat...
Name: SD - DD, dtype: object

In [46]:
X_train[0:6]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   15,  168,
        2136, 2137,   95,    6,   90,    4,    6,   15,   15,  168, 2136,
        2137,   95,    6,   90,    4, 

In [47]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [48]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)


13115


In [0]:
embedding_matrix = np.zeros((size_of_vocabulary, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Dense, Input, Flatten, BatchNormalization, Dropout
from keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Bidirectional
from keras.models import Model, Sequential
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

In [97]:
pip install keras_self_attention

In [0]:
from keras_self_attention import SeqSelfAttention

In [0]:
model=Sequential()
#embedding layer
#model.add(Input(shape=(150,)))
model.add(Embedding(size_of_vocabulary,100,weights=[embedding_matrix],input_length=150,trainable=True)) 

#Bilstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2)))
model.add(SeqSelfAttention(units = 32, attention_activation='relu'))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
#model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=0, mode="auto", min_delta=0.0001, cooldown=0)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)

In [55]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 100)          1311500   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 150, 256)          234496    
_________________________________________________________________
seq_self_attention_2 (SeqSel (None, 150, 256)          16449     
_________________________________________________________________
flatten_2 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 74)                2841674   
Total params: 4,404,119
Trainable params: 4,404,119
Non-trainable params: 0
_________________________________________________________________


In [56]:
historical = model.fit(X_train,y_train_keras,batch_size=32,epochs=50,validation_data=(X_valid,y_valid_keras),verbose=1,callbacks=[es,rlr,mc])

Train on 6141 samples, validate on 1084 samples
Epoch 1/50
6141/6141 [==============================] - 144s 23ms/step - loss: 2.1148 - acc: 0.5375 - val_loss: 1.6067 - val_acc: 0.6255

Epoch 00001: val_acc improved from -inf to 0.62546, saving model to best_model.h5
Epoch 2/50
6141/6141 [==============================] - 145s 24ms/step - loss: 1.5421 - acc: 0.6124 - val_loss: 1.3784 - val_acc: 0.6568

Epoch 00002: val_acc improved from 0.62546 to 0.65683, saving model to best_model.h5
Epoch 3/50
6141/6141 [==============================] - 140s 23ms/step - loss: 1.2566 - acc: 0.6606 - val_loss: 1.2877 - val_acc: 0.6688

Epoch 00003: val_acc improved from 0.65683 to 0.66882, saving model to best_model.h5
Epoch 4/50
6141/6141 [==============================] - 140s 23ms/step - loss: 1.0173 - acc: 0.7154 - val_loss: 1.3372 - val_acc: 0.6780

Epoch 00004: val_acc improved from 0.66882 to 0.67804, saving model to best_model.h5
Epoch 5/50
6141/6141 [==============================] - 139s 23

In [0]:
from keras.layers.embeddings import Embedding

from sklearn.model_selection import train_test_split

# Split the data
x_train_vec, x_test_vec, y_train_vec, y_test_vec = train_test_split(cleansed_data['SD - DD'], X, test_size=0.15, shuffle= True)
x_train_vec, x_valid_vec, y_train_vec, y_valid_vec = train_test_split(x_train_vec, y_train_vec, test_size=0.15, shuffle= True)

In [64]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_vec)
y_test_cate = to_categorical(y_train_vec)
print(y_train_cate.shape)

x_train_vec = x_train_vec.astype(str)
x_valid_vec = x_valid_vec.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)
#preparing vocabulary
tokenizer.fit_on_texts(x_train_vec)

#converting text into integer sequences
x_tr_seq_vec  = tokenizer.texts_to_sequences(x_train_vec) 
x_val_seq_vec = tokenizer.texts_to_sequences(x_valid_vec)

#padding to prepare sequences of same length
x_tr_seq_vec  = pad_sequences(x_tr_seq_vec, maxlen=200)
x_val_seq_vec = pad_sequences(x_val_seq_vec, maxlen=200)

(6141, 74, 2)


In [65]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

13275


In [0]:
from gensim.models.keyedvectors import KeyedVectors
PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/"
EMBEDDING_FILE = 'SO_vectors_200.bin'
EmbeddingFile = PROJECT_DIR+EMBEDDING_FILE
word2vec = KeyedVectors.load_word2vec_format(EmbeddingFile, binary=True)
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000

In [0]:
# create a weight matrix for words in training docs
EMBEDDING_DIM = 200
embedding_matrix_word2vec = np.zeros((size_of_vocabulary, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    if word in word2vec.vocab:
        embedding_matrix_word2vec[i] = word2vec.word_vec(word)

In [0]:
model=Sequential()
#embedding layer
#model.add(Input(shape=(150,)))
model.add(Embedding(size_of_vocabulary,200,weights=[embedding_matrix_word2vec],input_length=200,trainable=True)) 

#Bilstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2)))
model.add(SeqSelfAttention(units = 32, attention_activation='relu'))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
#model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=0, mode="auto", min_delta=0.0001, cooldown=0)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)

In [72]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 200)          2655000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200, 256)          336896    
_________________________________________________________________
seq_self_attention_3 (SeqSel (None, 200, 256)          16449     
_________________________________________________________________
flatten_3 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 74)                3788874   
Total params: 6,797,219
Trainable params: 6,797,219
Non-trainable params: 0
_________________________________________________________________


In [73]:
history = model.fit(x_tr_seq_vec,y_train_vec,batch_size=32,epochs=50,validation_data=(x_val_seq_vec,y_valid_vec),verbose=1,callbacks=[es,mc])

Train on 6141 samples, validate on 1084 samples
Epoch 1/50
6141/6141 [==============================] - 235s 38ms/step - loss: 1.9436 - acc: 0.5620 - val_loss: 1.4913 - val_acc: 0.6273

Epoch 00001: val_acc improved from -inf to 0.62731, saving model to best_model.h5
Epoch 2/50
6141/6141 [==============================] - 232s 38ms/step - loss: 1.4074 - acc: 0.6341 - val_loss: 1.4616 - val_acc: 0.6541

Epoch 00002: val_acc improved from 0.62731 to 0.65406, saving model to best_model.h5
Epoch 3/50
6141/6141 [==============================] - 232s 38ms/step - loss: 1.1004 - acc: 0.7002 - val_loss: 1.4972 - val_acc: 0.6494

Epoch 00003: val_acc did not improve from 0.65406
Epoch 4/50
6141/6141 [==============================] - 233s 38ms/step - loss: 0.8335 - acc: 0.7522 - val_loss: 1.4762 - val_acc: 0.6762

Epoch 00004: val_acc improved from 0.65406 to 0.67620, saving model to best_model.h5
Epoch 5/50
6141/6141 [==============================] - 232s 38ms/step - loss: 0.6409 - acc: 0.809

In [0]:

from sklearn.model_selection import train_test_split

# Split the data
x_train_fast, x_test_fast, y_train_fast, y_test_fast = train_test_split(cleansed_data['SD - DD'], X, test_size=0.15, shuffle= True)
x_train_fast, x_valid_fast, y_train_fast, y_valid_fast = train_test_split(x_train_fast, y_train_fast, test_size=0.15, shuffle= True)

In [104]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_fast)
y_test_cate = to_categorical(y_valid_fast)
print(y_train_cate.shape)

x_train_fast = x_train_fast.astype(str)
x_valid_fast = x_valid_fast.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(list(x_train_fast))

#converting text into integer sequences
x_tr_seq_fast  = tokenizer.texts_to_sequences(x_train_fast) 
x_val_seq_fast = tokenizer.texts_to_sequences(x_valid_fast)

#padding to prepare sequences of same length
x_tr_seq_fast  = pad_sequences(x_tr_seq_fast, maxlen=300)
x_val_seq_fast = pad_sequences(x_val_seq_fast, maxlen=300)

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

(6141, 74, 2)
13218


In [0]:
max_features = 200000 #17780 #using all unique words
embedding_dim = 300
num_classes = 74
batch_size = 32

embedding_path = "/content/drive/My Drive/Colab Notebooks/wiki-news-300d-1M.vec"

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

word_index = tokenizer.word_index
nb_words = len(word_index)
embedding_matrix_fast = np.zeros((nb_words + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix_fast[i] = embedding_vector

In [0]:
model=Sequential()
#embedding layer
#model.add(Input(shape=(150,)))
model.add(Embedding(size_of_vocabulary,300,weights=[embedding_matrix_fast],input_length=300,trainable=True)) 

#Bilstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2)))
model.add(SeqSelfAttention(units = 32, attention_activation='relu'))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
#model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=0, mode="auto", min_delta=0.0001, cooldown=0)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)


In [108]:
history = model.fit(x_tr_seq_fast,y_train_fast,batch_size=32,epochs=10,validation_data=(x_val_seq_fast,y_valid_fast),verbose=1,callbacks=[es,mc])

Train on 6141 samples, validate on 1084 samples
Epoch 1/10
6141/6141 [==============================] - 404s 66ms/step - loss: 0.1954 - acc: 0.9360 - val_loss: 2.5173 - val_acc: 0.6614

Epoch 00001: val_acc did not improve from 0.67528
Epoch 2/10
6141/6141 [==============================] - 405s 66ms/step - loss: 0.2100 - acc: 0.9334 - val_loss: 2.7515 - val_acc: 0.6513

Epoch 00002: val_acc did not improve from 0.67528
Epoch 3/10
6141/6141 [==============================] - 407s 66ms/step - loss: 0.2172 - acc: 0.9306 - val_loss: 2.6633 - val_acc: 0.6568

Epoch 00003: val_acc did not improve from 0.67528
Epoch 4/10
6141/6141 [==============================] - 409s 67ms/step - loss: 0.2193 - acc: 0.9316 - val_loss: 3.0527 - val_acc: 0.6577

Epoch 00004: val_acc did not improve from 0.67528
Epoch 5/10
6141/6141 [==============================] - 412s 67ms/step - loss: 0.2049 - acc: 0.9314 - val_loss: 3.0461 - val_acc: 0.6411

Epoch 00005: val_acc did not improve from 0.67528
Epoch 6/10
6

In [109]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_fast,y_valid_fast, batch_size=32)
print(val_acc)

ValueError: ignored